## A Simple NN with doc2vec Embedding

## Import Packages and File

In [527]:
import pandas as pd 
import nltk
import numpy as np
#load inthe NTLK stopwords to remove articles, preposition and other words that are not actionable
from nltk.corpus import stopwords
# This allows to create individual objects from a bog of words
from nltk.tokenize import word_tokenize, sent_tokenize
# Lemmatizer helps to reduce words to the base form
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

df=pd.read_csv('summer-products-with-rating-and-performance_2020-08.csv')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/jasmineli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jasmineli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jasmineli/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Title Doc2Vec

#### Implementation Doc: https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html

In [528]:
def process_sentence(sentence):
    new_tokens = word_tokenize(sentence)
    new_tokens = [t.lower() for t in new_tokens]
    new_tokens =[t for t in new_tokens if t not in stopwords.words('english')]
    new_tokens = [t for t in new_tokens if t.isalpha()]
    lemmatizer = WordNetLemmatizer()
    new_tokens =[lemmatizer.lemmatize(t) for t in new_tokens]
    return new_tokens

In [529]:
titles = df['title_orig'].tolist()
tokens = [process_sentence(t) for t in titles]
df['title_pre'] = tokens
df.head()

,title,title_orig,price,retail_price,currency_buyer,units_sold,uses_ad_boosts,rating,rating_count,rating_five_count,...,merchant_rating,merchant_id,merchant_has_profile_picture,merchant_profile_picture,product_url,product_picture,product_id,theme,crawl_month,title_pre
0,2020 Summer Vintage Flamingo Print Pajamas Se...,2020 Summer Vintage Flamingo Print Pajamas Se...,16.00,14,EUR,100,0,3.76,54,26.0,...,4.128521,595097d6a26f6e070cb878d1,0,NaN,https://www.wish.com/c/5e9ae51d43d6a96e303acdb0,https://contestimg.wish.com/api/webimage/5e9ae...,5e9ae51d43d6a96e303acdb0,summer,2020-08,"[summer, vintage, flamingo, print, pajama, set..."
1,SSHOUSE Summer Casual Sleeveless Soirée Party ...,Women's Casual Summer Sleeveless Sexy Mini Dress,8.00,22,EUR,20000,1,3.45,6135,2269.0,...,3.899673,56458aa03a698c35c9050988,0,NaN,https://www.wish.com/c/58940d436a0d3d5da4e95a38,https://contestimg.wish.com/api/webimage/58940...,58940d436a0d3d5da4e95a38,summer,2020-08,"[woman, casual, summer, sleeveless, sexy, mini..."
2,2020 Nouvelle Arrivée Femmes Printemps et Été ...,2020 New Arrival Women Spring and Summer Beach...,8.00,43,EUR,100,0,3.57,14,5.0,...,3.989831,5d464a1ffdf7bc44ee933c65,0,NaN,https://www.wish.com/c/5ea10e2c617580260d55310a,https://contestimg.wish.com/api/webimage/5ea10...,5ea10e2c617580260d55310a,summer,2020-08,"[new, arrival, woman, spring, summer, beach, w..."
3,Hot Summer Cool T-shirt pour les femmes Mode T...,Hot Summer Cool T Shirt for Women Fashion Tops...,8.00,8,EUR,5000,1,4.03,579,295.0,...,4.020435,58cfdefdacb37b556efdff7c,0,NaN,https://www.wish.com/c/5cedf17ad1d44c52c59e4aca,https://contestimg.wish.com/api/webimage/5cedf...,5cedf17ad1d44c52c59e4aca,summer,2020-08,"[hot, summer, cool, shirt, woman, fashion, top..."
4,Femmes Shorts d'été à lacets taille élastique ...,Women Summer Shorts Lace Up Elastic Waistband ...,2.72,3,EUR,100,1,3.10,20,6.0,...,4.001588,5ab3b592c3911a095ad5dadb,0,NaN,https://www.wish.com/c/5ebf5819ebac372b070b0e70,https://contestimg.wish.com/api/webimage/5ebf5...,5ebf5819ebac372b070b0e70,summer,2020-08,"[woman, summer, short, lace, elastic, waistban..."


In [530]:
def tag_data(sentences):
    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(sentences)]
    return tagged_data

In [531]:
data = tag_data(titles)

In [532]:
max_epochs = 100
vec_size = 20
alpha = 0.025

# dm=1 means ‘distributed memory’ (PV-DM) and dm =0 means ‘distributed bag of words’ (PV-DBOW). 
# Distributed Memory model preserves the word order in a document whereas Distributed Bag of words just 
# uses the bag of words approach, which doesn’t preserve any word order.
model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1) 

model.build_vocab(data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

In [533]:
test_data = word_tokenize("I love Siri".lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('71')
print(similar_doc)


# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.docvecs['3'])

V1_infer [ 0.02032866 -0.02796199 -0.02156443  0.00342751 -0.00071154 -0.00065618
  0.01527996 -0.01481971  0.02236171 -0.02088824  0.01114816 -0.00908459
  0.01513362  0.00118999  0.016719   -0.00039846  0.02455194 -0.01949126
  0.00690318 -0.00014208]
[('127', 0.6784767508506775), ('764', 0.6265950202941895), ('752', 0.6139605045318604), ('25', 0.5985579490661621), ('789', 0.5886136293411255), ('246', 0.5713376998901367), ('889', 0.5617882013320923), ('1144', 0.5601301789283752), ('436', 0.5425677299499512), ('422', 0.539798378944397)]
[ 0.01457734  0.04639842  0.02174927 -0.02099785  0.01120952 -0.02206228
  0.0288847   0.00915761 -0.01139381 -0.02940559 -0.04013542 -0.00426537
 -0.04469682 -0.04611818 -0.03969955  0.01084536 -0.0325047  -0.03894172
  0.01065583  0.01026331]


/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  
/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  # This is added back by InteractiveShellApp.init_path()


## Preparing Label Y (Low / High Sales)

In [534]:
label = [1 if sales > 200 else 0 for sales in df["units_sold"]]
df['high_sale'] = label

## Metadata

In [535]:
# product color
def main_color(s):
  main_color = {"red":"red", "white":"white", "pink":"pink", "yellow":"yellow", "green":"green", "blue":"blue", "wine":"red", "burgundy":"red", "black":"black", "navy":"navy", "orange":"orange", 
  "rose":"pink", "gray":"gray", "grey":"gray", "purple":"purple", "violet":"purple", "army":"green", "leopard":"orange", "ivory":"white", 
  "brown":"brown", "coffee":"brown", "camel":"beige", "tan":"brown", "nude":"beige", "khaki":"khaki", "apricot":"yellow", "camouflage":"green", "jasper":"red"}  # ordered by importance
  for key, value in main_color.items():
    if key in s:
        return value
    return "others"
product_color = df["product_color"]
product_color = [s.lower() if type(s) is str else 'nan' for s in product_color]
product_color = [main_color(s) for s in product_color]
from matplotlib import colors
product_color = [(-0.1,-0.1,-0.1,-0.1) if s == "others" else colors.to_rgba(s) for s in product_color]

df['product_color_rgb'] = [np.array(t) for t in product_color]

# log prices
df['log_price'] = [np.log(p) for p in df["price"]]
df['log_retail_price'] = [np.log(p) for p in df["retail_price"]]

# log merchant rating count
df['log_merchant_rating_count'] = np.log(df['merchant_rating_count'])

# urgent text
df['urgent'] = [1 if s == "Quantité limitée !" else 0 for s in df["urgency_text"]]

/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


### Feature Engineering 

In [536]:
data = df[["log_price", "log_retail_price", "uses_ad_boosts", "badges_count", "badge_local_product", 
           "badge_product_quality", "badge_fast_shipping", "urgent", 
           "high_sale"]]
# generating doc2vec encoding for preprossed titles
df['doc2vec'] = [model.infer_vector(title) for title in df['title_pre']]
doc2vec = np.stack(df['doc2vec'].values, axis=0)
data2 = pd.DataFrame()
for i in range(20):
    data["doc2vec"+str(i)] = doc2vec[:,i]
    # data for only using titile text as input
    data2["doc2vec"+str(i)] = doc2vec[:,i]
rgb = df["product_color_rgb"]
rgb = np.stack(rgb.values, axis=0)
for i in range(4):
    data["product_color_rgb"+str(i)] = rgb[:,i]

/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [537]:
data.replace([np.inf, -np.inf], np.nan, inplace=True)

/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


### 1. Building NN Model with Doc2vec

In [538]:
# X = data.loc[:, ~data.columns.isin(['high_sale'])]
X = data2
y = data['high_sale']

In [539]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from keras.layers import Input, Dense, Concatenate
from keras.models import Model
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.125, random_state=42)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.14286, random_state=42)

In [540]:
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of dev examples = " + str(X_dev.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(y_train.shape))
print ("X_dev shape: " + str(X_dev.shape))
print ("Y_dev shape: " + str(y_dev.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(y_test.shape))

number of training examples = 1179
number of dev examples = 197
number of test examples = 197
X_train shape: (1179, 20)
Y_train shape: (1179,)
X_dev shape: (197, 20)
Y_dev shape: (197,)
X_test shape: (197, 20)
Y_test shape: (197,)


In [541]:
# from sklearn.linear_model import LogisticRegression

# classifier = LogisticRegression()
# classifier.fit(X_train, y_train)

# score = classifier.score(X_test, y_test)

# print("Accuracy:", score)

In [542]:
X_train = tf.expand_dims(X_train, axis=-1)
input_shape = X_train.shape[1:]

In [543]:
# define model
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [544]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

In [545]:
history = model.fit(
    X_train,
    y_train,
    epochs=1000,
    verbose=2,
    validation_data=(X_dev, y_dev),
    batch_size=32,
    callbacks=[es]
)
model.summary()

Epoch 1/1000


/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


37/37 - 1s - loss: 0.6831 - accuracy: 0.5759 - val_loss: 0.6653 - val_accuracy: 0.6345 - 783ms/epoch - 21ms/step
Epoch 2/1000
37/37 - 0s - loss: 0.6818 - accuracy: 0.5827 - val_loss: 0.6606 - val_accuracy: 0.6345 - 83ms/epoch - 2ms/step


/Users/jasmineli/opt/anaconda3/envs/etsy/lib/python3.7/site-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


Epoch 3/1000
37/37 - 0s - loss: 0.6815 - accuracy: 0.5827 - val_loss: 0.6631 - val_accuracy: 0.6345 - 78ms/epoch - 2ms/step
Epoch 4/1000
37/37 - 0s - loss: 0.6812 - accuracy: 0.5827 - val_loss: 0.6667 - val_accuracy: 0.6345 - 61ms/epoch - 2ms/step
Epoch 5/1000
37/37 - 0s - loss: 0.6805 - accuracy: 0.5827 - val_loss: 0.6618 - val_accuracy: 0.6345 - 59ms/epoch - 2ms/step
Epoch 6/1000
37/37 - 0s - loss: 0.6791 - accuracy: 0.5827 - val_loss: 0.6678 - val_accuracy: 0.6345 - 63ms/epoch - 2ms/step
Epoch 7/1000
37/37 - 0s - loss: 0.6795 - accuracy: 0.5827 - val_loss: 0.6625 - val_accuracy: 0.6345 - 63ms/epoch - 2ms/step
Epoch 8/1000
37/37 - 0s - loss: 0.6789 - accuracy: 0.5827 - val_loss: 0.6628 - val_accuracy: 0.6345 - 61ms/epoch - 2ms/step
Epoch 9/1000
37/37 - 0s - loss: 0.6795 - accuracy: 0.5827 - val_loss: 0.6651 - val_accuracy: 0.6345 - 61ms/epoch - 2ms/step
Epoch 10/1000
37/37 - 0s - loss: 0.6800 - accuracy: 0.5827 - val_loss: 0.6618 - val_accuracy: 0.6345 - 61ms/epoch - 2ms/step
Epoch 1

...

Epoch 135/1000
37/37 - 0s - loss: 0.6611 - accuracy: 0.6098 - val_loss: 0.6552 - val_accuracy: 0.6345 - 60ms/epoch - 2ms/step
Epoch 136/1000
37/37 - 0s - loss: 0.6579 - accuracy: 0.6149 - val_loss: 0.6569 - val_accuracy: 0.6193 - 60ms/epoch - 2ms/step
Epoch 137/1000
37/37 - 0s - loss: 0.6574 - accuracy: 0.6056 - val_loss: 0.6581 - val_accuracy: 0.6091 - 59ms/epoch - 2ms/step
Epoch 138/1000
37/37 - 0s - loss: 0.6558 - accuracy: 0.6064 - val_loss: 0.6539 - val_accuracy: 0.6294 - 60ms/epoch - 2ms/step
Epoch 139/1000
37/37 - 0s - loss: 0.6597 - accuracy: 0.5954 - val_loss: 0.6554 - val_accuracy: 0.6497 - 62ms/epoch - 2ms/step
Epoch 140/1000
37/37 - 0s - loss: 0.6576 - accuracy: 0.5997 - val_loss: 0.6658 - val_accuracy: 0.5736 - 60ms/epoch - 2ms/step
Epoch 141/1000
37/37 - 0s - loss: 0.6580 - accuracy: 0.6115 - val_loss: 0.6566 - val_accuracy: 0.6041 - 59ms/epoch - 2ms/step
Epoch 142/1000
37/37 - 0s - loss: 0.6524 - accuracy: 0.6124 - val_loss: 0.6546 - val_accuracy: 0.6091 - 60ms/epoch - 2

37/37 - 0s - loss: 0.6448 - accuracy: 0.6115 - val_loss: 0.6484 - val_accuracy: 0.6294 - 60ms/epoch - 2ms/step
Epoch 201/1000
37/37 - 0s - loss: 0.6379 - accuracy: 0.6226 - val_loss: 0.6565 - val_accuracy: 0.6041 - 59ms/epoch - 2ms/step
Epoch 202/1000
37/37 - 0s - loss: 0.6448 - accuracy: 0.6081 - val_loss: 0.6462 - val_accuracy: 0.6345 - 61ms/epoch - 2ms/step
Epoch 203/1000
37/37 - 0s - loss: 0.6406 - accuracy: 0.6302 - val_loss: 0.6460 - val_accuracy: 0.6447 - 61ms/epoch - 2ms/step
Epoch 204/1000
37/37 - 0s - loss: 0.6463 - accuracy: 0.6387 - val_loss: 0.6432 - val_accuracy: 0.6345 - 59ms/epoch - 2ms/step
Epoch 205/1000
37/37 - 0s - loss: 0.6468 - accuracy: 0.6192 - val_loss: 0.6457 - val_accuracy: 0.6447 - 60ms/epoch - 2ms/step
Epoch 206/1000
37/37 - 0s - loss: 0.6397 - accuracy: 0.6175 - val_loss: 0.6479 - val_accuracy: 0.6345 - 61ms/epoch - 2ms/step
Epoch 207/1000
37/37 - 0s - loss: 0.6452 - accuracy: 0.6175 - val_loss: 0.6464 - val_accuracy: 0.6193 - 60ms/epoch - 2ms/step
Epoch 2

Epoch 266/1000
37/37 - 0s - loss: 0.6304 - accuracy: 0.6404 - val_loss: 0.6418 - val_accuracy: 0.6447 - 59ms/epoch - 2ms/step
Epoch 267/1000
37/37 - 0s - loss: 0.6291 - accuracy: 0.6387 - val_loss: 0.6378 - val_accuracy: 0.6650 - 58ms/epoch - 2ms/step
Epoch 268/1000
37/37 - 0s - loss: 0.6276 - accuracy: 0.6438 - val_loss: 0.6394 - val_accuracy: 0.6447 - 59ms/epoch - 2ms/step
Epoch 269/1000
37/37 - 0s - loss: 0.6307 - accuracy: 0.6446 - val_loss: 0.6371 - val_accuracy: 0.6650 - 60ms/epoch - 2ms/step
Epoch 270/1000
37/37 - 0s - loss: 0.6272 - accuracy: 0.6463 - val_loss: 0.6415 - val_accuracy: 0.6193 - 59ms/epoch - 2ms/step
Epoch 271/1000
37/37 - 0s - loss: 0.6219 - accuracy: 0.6556 - val_loss: 0.6391 - val_accuracy: 0.6497 - 59ms/epoch - 2ms/step
Epoch 272/1000
37/37 - 0s - loss: 0.6238 - accuracy: 0.6412 - val_loss: 0.6526 - val_accuracy: 0.5685 - 60ms/epoch - 2ms/step
Epoch 273/1000
37/37 - 0s - loss: 0.6454 - accuracy: 0.6209 - val_loss: 0.6361 - val_accuracy: 0.6396 - 63ms/epoch - 2

37/37 - 0s - loss: 0.6076 - accuracy: 0.6624 - val_loss: 0.6473 - val_accuracy: 0.5990 - 63ms/epoch - 2ms/step
Epoch 332/1000
37/37 - 0s - loss: 0.6232 - accuracy: 0.6361 - val_loss: 0.6397 - val_accuracy: 0.6294 - 59ms/epoch - 2ms/step
Epoch 333/1000
37/37 - 0s - loss: 0.6099 - accuracy: 0.6514 - val_loss: 0.6585 - val_accuracy: 0.5482 - 60ms/epoch - 2ms/step
Epoch 334/1000
37/37 - 0s - loss: 0.6103 - accuracy: 0.6480 - val_loss: 0.6372 - val_accuracy: 0.6497 - 60ms/epoch - 2ms/step
Epoch 335/1000
37/37 - 0s - loss: 0.6136 - accuracy: 0.6675 - val_loss: 0.6402 - val_accuracy: 0.6294 - 60ms/epoch - 2ms/step
Epoch 336/1000
37/37 - 0s - loss: 0.6143 - accuracy: 0.6497 - val_loss: 0.6331 - val_accuracy: 0.6447 - 60ms/epoch - 2ms/step
Epoch 337/1000
37/37 - 0s - loss: 0.6084 - accuracy: 0.6667 - val_loss: 0.6391 - val_accuracy: 0.6396 - 62ms/epoch - 2ms/step
Epoch 338/1000
37/37 - 0s - loss: 0.6197 - accuracy: 0.6489 - val_loss: 0.6341 - val_accuracy: 0.6548 - 62ms/epoch - 2ms/step
Epoch 3

Epoch 397/1000
37/37 - 0s - loss: 0.5962 - accuracy: 0.6811 - val_loss: 0.6461 - val_accuracy: 0.6396 - 60ms/epoch - 2ms/step
Epoch 00397: early stopping
Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_27 (Conv1D)          (None, 13, 32)            288       
                                                                 
 max_pooling1d_16 (MaxPoolin  (None, 6, 32)            0         
 g1D)                                                            
                                                                 
 dropout_29 (Dropout)        (None, 6, 32)             0         
                                                                 
 flatten_20 (Flatten)        (None, 192)               0         
                                                                 
 dense_50 (Dense)            (None, 128)               24704     
                               

### LogisticRegression